In [3]:
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.keys import Keys
import re
from time import sleep
from datetime import datetime
from openpyxl import Workbook
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\dmxdu\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [37]:
# setup driver
url = 'https://steamcommunity.com/app/730/reviews/?browsefilter=toprated'
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(options=options, executable_path=r'C:\Users\dmxdu\AppData\Local\Programs\Common\chromedriver.exe')
driver.get(url)

# get current position of y scrollbar
last_position = driver.execute_script("return window.pageYOffset;")

reviews = []
review_ids = set()
running = True

stop = 0
while stop < 200:
    # get cards on the page
    cards = driver.find_elements_by_class_name('apphub_Card')

    for card in cards[-20:]:  # only the tail end are new cards

        # gamer profile url
        profile_url = card.find_element_by_xpath('.//div[@class="apphub_friend_block"]/div/a[2]').get_attribute('href')

        # Number of other products
        num_prod = card.find_element_by_xpath('.//div[@class="apphub_CardContentMoreLink ellipsis"]').text

        # steam id
        steam_id = profile_url.split('/')[-2]
        
        # check to see if I've already collected this review
        if steam_id in review_ids:
            continue
        else:
            review_ids.add(steam_id)

        # username
        user_name = card.find_element_by_xpath('.//div[@class="apphub_friend_block"]/div/a[2]').text

        # language of the review
        date_posted = card.find_element_by_xpath('.//div[@class="apphub_CardTextContent"]/div').text
        review_content = card.find_element_by_xpath('.//div[@class="apphub_CardTextContent"]').text.replace(date_posted,'').strip()    

        # review length
        review_length = len(review_content.replace(' ', ''))    

        # recommendation
        thumb_text = card.find_element_by_xpath('.//div[@class="reviewInfo"]/div[2]').text
        thumb_text    

        # amount of play hours
        play_hours = card.find_element_by_xpath('.//div[@class="reviewInfo"]/div[3]').text
        play_hours    

        # save review
        review = (steam_id, num_prod, review_content, thumb_text, review_length, play_hours, date_posted)
        reviews.append(review)    
        
    # attempt to scroll down thrice.. then break
    scroll_attempt = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
        sleep(0.5)
        curr_position = driver.execute_script("return window.pageYOffset;")
        
        if curr_position == last_position:
            scroll_attempt += 1
            sleep(0.5)
            
            if curr_position >= 3:
                running = False
                break
        else:
            last_position = curr_position
            break  # continue scraping the results
    stop += 1

# shutdown the web driver
driver.close()

In [38]:
# save the file to a CSV file
with open(f'Steam_Reviews_CSGO.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['SteamId', 'Num of Products', 'ReviewText', 'Review', 'ReviewLength(Chars)', 'PlayHours', 'DatePosted'])
    writer.writerows(reviews)